In [1]:
import requests # The requests library 
import bs4 as bs # BeautifulSoup4 is a Python library 
import pandas as pd
from collections import Counter
import re

In [39]:
df =  pd.read_csv('airbnb-listings.csv', sep=',', engine='python')
df = pd.DataFrame(df)
df.head()

,ID,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,...,Review Scores Communication,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features
0,NaN,https://www.airbnb.com/rooms/2105840,NaN,4/2/17,Room w/priv. bath Duboce Triangle 1,Stay with us in our live/work space and become...,Stay with us in our live/work space and become...,Stay with us in our live/work space and become...,none,We think that this is the best neighborhood in...,...,10.0,10.0,9.0,NaN,SAN FRANCISCO,strict,2,1.16,NaN,NaN
1,NaN,https://www.airbnb.com/rooms/5505643,NaN,4/2/17,Modern Mission-Dolores apartment,"Top floor apartment in a two unit building, bu...",Full open floor living room/dining room/kitche...,"Top floor apartment in a two unit building, bu...",none,Fantastic location at the intersection of Miss...,...,10.0,10.0,10.0,NaN,SAN FRANCISCO,strict,1,2.51,NaN,NaN
2,NaN,https://www.airbnb.com/rooms/746079,NaN,4/2/17,Castro/Corbett Heights Condo,Enjoy this quiet bedroom suite in my luxury Co...,Enjoy this quiet bedroom suite in my luxury Co...,Enjoy this quiet bedroom suite in my luxury Co...,none,"Locals call my neighborhood ""Corbett Heights"" ...",...,10.0,10.0,9.0,SAN FRANCISCO SHORT-TERM RENTAL REGISTRATION N...,SAN FRANCISCO,flexible,1,4.30,NaN,NaN
3,NaN,https://www.airbnb.com/rooms/11680405,NaN,4/2/17,"Sun-filled, Spacious Duboce 1BR","Our airy, spacious 1 BR in the Duboce/Castro a...",Both the bedroom and living room have bay wind...,"Our airy, spacious 1 BR in the Duboce/Castro a...",none,This is the best situated neighborhood in SF. ...,...,10.0,10.0,10.0,NaN,SAN FRANCISCO,moderate,1,0.42,NaN,NaN
4,NaN,https://www.airbnb.com/rooms/1304068,NaN,4/2/17,Historic SF Residence on a park,"Central location, well maintained Victorian, g...","Central location, well maintained Victorian, g...","Central location, well maintained Victorian, g...",none,NaN,...,10.0,10.0,10.0,STR-0001041,SAN FRANCISCO,moderate,1,0.23,NaN,NaN


In [40]:
# example of expired url 
source = requests.get("https://www.airbnb.com/rooms/841083") 
soup = bs.BeautifulSoup(source.content, features='html.parser')
#print(soup)
title = soup.find('title')
print(title)

<title>Vacation Rentals, Homes, Experiences &amp; Places - Airbnb</title>


From data set, we found some url expired in Listing Url column so we can't get price information by web scraping.
Let's keep non-expire url data set only.

In [41]:
# check the website is expired so we can't get price information ( expire = 1 , otherwise = 0)
expire = list()
for i in df['Listing Url']:
    source = requests.get(i) 
    soup = bs.BeautifulSoup(source.content, features='html.parser')
    title = soup.find('title')
    if title.text == ("Vacation Rentals, Homes, Experiences & Places - Airbnb"):
        expire.append(1)
    else:
        expire.append(0)     

ConnectionError: HTTPSConnectionPool(host='www.airbnb.com', port=443): Max retries exceeded with url: /rooms/3310076 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001A6563A3748>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))

In [28]:
# total nrow of data set
len(expire)

2948

In [29]:
# total number of non-expired url
expire.count(0)

2866

In [30]:
# add column "expire" in df
df['expire']=expire

In [32]:
df.head(5)

,Unnamed: 0,ID,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,...,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features,expire
0,0,NaN,https://www.airbnb.com/rooms/2105840,NaN,4/2/17,Room w/priv. bath Duboce Triangle 1,Stay with us in our live/work space and become...,Stay with us in our live/work space and become...,Stay with us in our live/work space and become...,none,...,10.0,9.0,NaN,SAN FRANCISCO,strict,2,1.16,NaN,NaN,0
1,1,NaN,https://www.airbnb.com/rooms/5505643,NaN,4/2/17,Modern Mission-Dolores apartment,"Top floor apartment in a two unit building, bu...",Full open floor living room/dining room/kitche...,"Top floor apartment in a two unit building, bu...",none,...,10.0,10.0,NaN,SAN FRANCISCO,strict,1,2.51,NaN,NaN,0
2,2,NaN,https://www.airbnb.com/rooms/746079,NaN,4/2/17,Castro/Corbett Heights Condo,Enjoy this quiet bedroom suite in my luxury Co...,Enjoy this quiet bedroom suite in my luxury Co...,Enjoy this quiet bedroom suite in my luxury Co...,none,...,10.0,9.0,SAN FRANCISCO SHORT-TERM RENTAL REGISTRATION N...,SAN FRANCISCO,flexible,1,4.30,NaN,NaN,0
3,3,NaN,https://www.airbnb.com/rooms/1304068,NaN,4/2/17,Historic SF Residence on a park,"Central location, well maintained Victorian, g...","Central location, well maintained Victorian, g...","Central location, well maintained Victorian, g...",none,...,10.0,10.0,STR-0001041,SAN FRANCISCO,moderate,1,0.23,NaN,NaN,0
4,4,NaN,https://www.airbnb.com/rooms/3116428,NaN,4/2/17,Sunny/stylish/central/quiet SF 2BR,"Sunny, stylish top floor apartment in a recent...",The apartment is located on the top floor of a...,"Sunny, stylish top floor apartment in a recent...",none,...,10.0,10.0,STR-0000683,SAN FRANCISCO,moderate,1,0.56,NaN,NaN,0


In [33]:
# drop expire-website(expire = 1) and keep non-expire one (expire = 0)
df = df[df.expire != 1]

In [34]:
df.head(5)

,Unnamed: 0,ID,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,...,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features,expire
0,0,NaN,https://www.airbnb.com/rooms/2105840,NaN,4/2/17,Room w/priv. bath Duboce Triangle 1,Stay with us in our live/work space and become...,Stay with us in our live/work space and become...,Stay with us in our live/work space and become...,none,...,10.0,9.0,NaN,SAN FRANCISCO,strict,2,1.16,NaN,NaN,0
1,1,NaN,https://www.airbnb.com/rooms/5505643,NaN,4/2/17,Modern Mission-Dolores apartment,"Top floor apartment in a two unit building, bu...",Full open floor living room/dining room/kitche...,"Top floor apartment in a two unit building, bu...",none,...,10.0,10.0,NaN,SAN FRANCISCO,strict,1,2.51,NaN,NaN,0
2,2,NaN,https://www.airbnb.com/rooms/746079,NaN,4/2/17,Castro/Corbett Heights Condo,Enjoy this quiet bedroom suite in my luxury Co...,Enjoy this quiet bedroom suite in my luxury Co...,Enjoy this quiet bedroom suite in my luxury Co...,none,...,10.0,9.0,SAN FRANCISCO SHORT-TERM RENTAL REGISTRATION N...,SAN FRANCISCO,flexible,1,4.30,NaN,NaN,0
3,3,NaN,https://www.airbnb.com/rooms/1304068,NaN,4/2/17,Historic SF Residence on a park,"Central location, well maintained Victorian, g...","Central location, well maintained Victorian, g...","Central location, well maintained Victorian, g...",none,...,10.0,10.0,STR-0001041,SAN FRANCISCO,moderate,1,0.23,NaN,NaN,0
4,4,NaN,https://www.airbnb.com/rooms/3116428,NaN,4/2/17,Sunny/stylish/central/quiet SF 2BR,"Sunny, stylish top floor apartment in a recent...",The apartment is located on the top floor of a...,"Sunny, stylish top floor apartment in a recent...",none,...,10.0,10.0,STR-0000683,SAN FRANCISCO,moderate,1,0.56,NaN,NaN,0


In [35]:
#df.to_csv('clean-airbnb.csv', sep='\t')

In [36]:
len(df)

2866

In [38]:
# get price informatio from airbnb url (webscraping)
Price = list()
for i in df['Listing Url']:
    source = requests.get(i) 
    soup = bs.BeautifulSoup(source.text)
    p = soup.find(attrs={'name':'description'})
    p = p['content'].split()[7]
    p = p.replace('.',"")
    p = p.replace('$',"")
    p = int(p) # convert string to int (price)
    Price.append(p)
    
df['Price']= Price # add price array in dataframe      

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


TypeError: 'NoneType' object is not subscriptable

In [ ]:
df.to_csv('data/clean-airbnb.csv', sep='\t')